# Lab 13: Database Triggers and Events - Interactive Practice

This notebook provides hands-on practice with MySQL triggers and scheduled events. You'll learn to create automated database operations that respond to data changes and schedule recurring tasks.

## Learning Objectives
- Create and manage database triggers (BEFORE/AFTER, INSERT/UPDATE/DELETE)
- Implement business logic through triggers
- Work with scheduled events and automation
- Handle trigger execution order and conflicts
- Debug and troubleshoot trigger issues
- Apply best practices for trigger development

## Prerequisites
- Lab 12: Stored Procedures and Views
- MySQL Server running
- mysql-connector-python installed

## Setup

In [ ]:
# Install required packages if not already installed
# !pip install mysql-connector-python pandas

# Import required libraries
import mysql.connector
from mysql.connector import Error
import pandas as pd
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Database connection configuration
config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'your_password',  # Replace with your MySQL password
    'database': 'triggers_events_db',
    'autocommit': True  # Enable autocommit for trigger demonstrations
}

def create_connection():
    """Create database connection"""
    try:
        connection = mysql.connector.connect(**config)
        print("✅ Connected to MySQL database")
        return connection
    except Error as e:
        print(f"❌ Error connecting to MySQL: {e}")
        return None

def execute_query(connection, query, description="", fetch=True):
    """Execute a query and optionally return results as DataFrame"""
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        
        if description:
            print(f"\n{description}")
        
        if fetch and cursor.description:
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()
            cursor.close()
            
            if rows:
                df = pd.DataFrame(rows, columns=columns)
                return df
            else:
                print("No results returned")
                return None
        else:
            cursor.close()
            print("Query executed successfully")
            return None
            
    except Error as e:
        print(f"❌ Error executing query: {e}")
        return None

# Test connection
conn = create_connection()
if conn:
    conn.close()
    print("✅ Connection test successful")
else:
    print("❌ Please check your database configuration")

## Exercise 1: Setting Up the Database

Let's create the database and tables for our trigger examples.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create database
    execute_query(conn, "CREATE DATABASE IF NOT EXISTS triggers_events_db", 
                  description="Creating triggers_events_db database")
    
    # Switch to the database
    execute_query(conn, "USE triggers_events_db", 
                  description="Switching to triggers_events_db")
    
    # Create employees table
    create_employees_sql = """
    CREATE TABLE IF NOT EXISTS employees (
        id INT PRIMARY KEY AUTO_INCREMENT,
        name VARCHAR(100) NOT NULL,
        department VARCHAR(50),
        salary DECIMAL(10,2),
        hire_date DATE,
        last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    execute_query(conn, create_employees_sql, 
                  description="Creating employees table")
    
    # Create audit_log table
    create_audit_sql = """
    CREATE TABLE IF NOT EXISTS audit_log (
        id INT PRIMARY KEY AUTO_INCREMENT,
        table_name VARCHAR(50),
        operation VARCHAR(10),
        record_id INT,
        old_values JSON,
        new_values JSON,
        user_name VARCHAR(100),
        operation_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    execute_query(conn, create_audit_sql, 
                  description="Creating audit_log table")
    
    # Create departments table
    create_departments_sql = """
    CREATE TABLE IF NOT EXISTS departments (
        id INT PRIMARY KEY AUTO_INCREMENT,
        name VARCHAR(50) UNIQUE NOT NULL,
        budget DECIMAL(12,2),
        current_spending DECIMAL(12,2) DEFAULT 0,
        employee_count INT DEFAULT 0,
        max_employees INT DEFAULT 50
    )
    """
    execute_query(conn, create_departments_sql, 
                  description="Creating departments table")
    
    # Insert sample departments
    insert_departments_sql = """
    INSERT IGNORE INTO departments (name, budget, max_employees) VALUES
    ('IT', 500000.00, 20),
    ('HR', 300000.00, 15),
    ('Sales', 400000.00, 25),
    ('Marketing', 350000.00, 18),
    ('Finance', 450000.00, 12)
    """
    execute_query(conn, insert_departments_sql, 
                  description="Inserting sample departments")
    
    print("\n✅ Database setup completed!")
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 2: Basic Audit Triggers

Let's create and test basic audit triggers that log all changes to the employees table.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create audit triggers
    audit_triggers_sql = """
    DELIMITER $$
    
    CREATE TRIGGER audit_employee_insert
    AFTER INSERT ON employees
    FOR EACH ROW
    BEGIN
        INSERT INTO audit_log (table_name, operation, record_id, new_values, user_name)
        VALUES ('employees', 'INSERT', NEW.id,
                JSON_OBJECT('name', NEW.name, 'department', NEW.department,
                            'salary', NEW.salary, 'hire_date', NEW.hire_date),
                USER());
    END$$
    
    CREATE TRIGGER audit_employee_update
    AFTER UPDATE ON employees
    FOR EACH ROW
    BEGIN
        INSERT INTO audit_log (table_name, operation, record_id,
                              old_values, new_values, user_name)
        VALUES ('employees', 'UPDATE', NEW.id,
                JSON_OBJECT('name', OLD.name, 'department', OLD.department,
                            'salary', OLD.salary, 'hire_date', OLD.hire_date),
                JSON_OBJECT('name', NEW.name, 'department', NEW.department,
                            'salary', NEW.salary, 'hire_date', NEW.hire_date),
                USER());
    END$$
    
    CREATE TRIGGER audit_employee_delete
    AFTER DELETE ON employees
    FOR EACH ROW
    BEGIN
        INSERT INTO audit_log (table_name, operation, record_id, old_values, user_name)
        VALUES ('employees', 'DELETE', OLD.id,
                JSON_OBJECT('name', OLD.name, 'department', OLD.department,
                            'salary', OLD.salary, 'hire_date', OLD.hire_date),
                USER());
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, audit_triggers_sql, 
                  description="Creating audit triggers", fetch=False)
    
    # Test INSERT trigger
    insert_sql = """
    INSERT INTO employees (name, department, salary, hire_date) VALUES
    ('John Doe', 'IT', 75000.00, '2024-01-15'),
    ('Jane Smith', 'HR', 65000.00, '2024-02-20')
    """
    execute_query(conn, insert_sql, 
                  description="Testing INSERT trigger", fetch=False)
    
    # Test UPDATE trigger
    update_sql = "UPDATE employees SET salary = 80000.00 WHERE name = 'John Doe'"
    execute_query(conn, update_sql, 
                  description="Testing UPDATE trigger", fetch=False)
    
    # Test DELETE trigger
    delete_sql = "DELETE FROM employees WHERE name = 'Jane Smith'"
    execute_query(conn, delete_sql, 
                  description="Testing DELETE trigger", fetch=False)
    
    # Check audit log
    df = execute_query(conn, "SELECT * FROM audit_log ORDER BY operation_time DESC",
                      description="Checking audit log entries")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 3: BEFORE Triggers for Validation

Let's create BEFORE triggers that validate data and enforce business rules.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create validation triggers
    validation_triggers_sql = """
    DELIMITER $$
    
    CREATE TRIGGER validate_employee_before_insert
    BEFORE INSERT ON employees
    FOR EACH ROW
    BEGIN
        -- Validate salary is positive
        IF NEW.salary <= 0 THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Salary must be greater than 0';
        END IF;
        
        -- Validate hire date is not in the future
        IF NEW.hire_date > CURDATE() THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Hire date cannot be in the future';
        END IF;
        
        -- Set default department if not provided
        IF NEW.department IS NULL OR NEW.department = '' THEN
            SET NEW.department = 'General';
        END IF;
        
        -- Capitalize name
        SET NEW.name = TRIM(UPPER(NEW.name));
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, validation_triggers_sql, 
                  description="Creating validation triggers", fetch=False)
    
    # Test valid insert
    valid_insert_sql = """
    INSERT INTO employees (name, department, salary, hire_date) VALUES
    ('alice johnson', 'IT', 70000.00, '2024-03-10')
    """
    execute_query(conn, valid_insert_sql, 
                  description="Testing valid INSERT (should succeed)", fetch=False)
    
    # Check the result
    df = execute_query(conn, "SELECT name, department, salary FROM employees WHERE name LIKE 'ALICE%',
                      description="Checking data cleaning results")
    if df is not None:
        display(df)
    
    # Test invalid inserts (these should fail)
    print("\nTesting invalid inserts (should fail):")
    
    invalid_tests = [
        ("Negative salary", "INSERT INTO employees (name, department, salary, hire_date) VALUES ('Test User', 'IT', -50000.00, '2024-01-01')"),
        ("Future hire date", "INSERT INTO employees (name, department, salary, hire_date) VALUES ('Future Employee', 'IT', 60000.00, '2025-01-01')")
    ]
    
    for test_name, test_sql in invalid_tests:
        print(f"\n🧪 Testing: {test_name}")
        try:
            execute_query(conn, test_sql, fetch=False)
            print("❌ Expected this to fail!")
        except Exception as e:
            print(f"✅ Correctly rejected: {str(e)}")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 4: Salary Tracking Triggers

Let's create triggers that track salary changes and enforce business rules.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create salary_history table first
    create_salary_history_sql = """
    CREATE TABLE IF NOT EXISTS salary_history (
        id INT PRIMARY KEY AUTO_INCREMENT,
        employee_id INT,
        old_salary DECIMAL(10,2),
        new_salary DECIMAL(10,2),
        change_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        reason VARCHAR(255),
        FOREIGN KEY (employee_id) REFERENCES employees(id)
    )
    """
    execute_query(conn, create_salary_history_sql, 
                  description="Creating salary_history table")
    
    # Create notifications table
    create_notifications_sql = """
    CREATE TABLE IF NOT EXISTS notifications (
        id INT PRIMARY KEY AUTO_INCREMENT,
        message TEXT,
        recipient VARCHAR(100),
        sent_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status ENUM('pending', 'sent', 'failed') DEFAULT 'pending'
    )
    """
    execute_query(conn, create_notifications_sql, 
                  description="Creating notifications table")
    
    # Create salary tracking trigger
    salary_trigger_sql = """
    DELIMITER $$
    
    CREATE TRIGGER track_salary_changes
    BEFORE UPDATE ON employees
    FOR EACH ROW
    BEGIN
        -- Only track if salary actually changed
        IF OLD.salary != NEW.salary THEN
            INSERT INTO salary_history (employee_id, old_salary, new_salary, reason)
            VALUES (OLD.id, OLD.salary, NEW.salary, 'Salary update via trigger');
            
            -- Prevent salary decreases over 20%
            IF NEW.salary < OLD.salary * 0.8 THEN
                SIGNAL SQLSTATE '45000'
                SET MESSAGE_TEXT = 'Salary cannot be decreased by more than 20%';
            END IF;
            
            -- Notify for large salary increases
            IF NEW.salary > OLD.salary * 1.5 THEN
                INSERT INTO notifications (message, recipient)
                VALUES (CONCAT('Large salary increase for ', NEW.name,
                              ': $', OLD.salary, ' → $', NEW.salary),
                        'hr@company.com');
            END IF;
        END IF;
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, salary_trigger_sql, 
                  description="Creating salary tracking trigger", fetch=False)
    
    # Test salary updates
    salary_tests = [
        ("Normal increase", "UPDATE employees SET salary = 75000.00 WHERE name = 'ALICE JOHNSON'", True),
        ("Large increase", "UPDATE employees SET salary = 120000.00 WHERE name = 'JOHN DOE'", True),
        ("Excessive decrease", "UPDATE employees SET salary = 30000.00 WHERE name = 'JOHN DOE'", False)
    ]
    
    for test_name, test_sql, should_succeed in salary_tests:
        print(f"\n🧪 Testing: {test_name}")
        try:
            execute_query(conn, test_sql, fetch=False)
            if should_succeed:
                print("✅ Succeeded as expected")
            else:
                print("❌ Expected this to fail!")
        except Exception as e:
            if not should_succeed:
                print(f"✅ Correctly rejected: {str(e)}")
            else:
                print(f"❌ Unexpected failure: {str(e)}")
    
    # Check salary history
    df = execute_query(conn, "SELECT * FROM salary_history ORDER BY change_date DESC",
                      description="Checking salary history")
    if df is not None:
        display(df)
    
    # Check notifications
    df = execute_query(conn, "SELECT * FROM notifications ORDER BY sent_at DESC",
                      description="Checking notifications")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 5: Department Budget Management

Let's create triggers that manage department budgets and employee limits.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create department budget triggers
    budget_triggers_sql = """
    DELIMITER $$
    
    CREATE TRIGGER check_department_budget_before_insert
    BEFORE INSERT ON employees
    FOR EACH ROW
    BEGIN
        DECLARE dept_budget DECIMAL(12,2);
        DECLARE dept_spending DECIMAL(12,2);
        DECLARE dept_max_emp INT;
        DECLARE dept_current_emp INT;
        
        -- Get department information
        SELECT budget, current_spending, max_employees, employee_count
        INTO dept_budget, dept_spending, dept_max_emp, dept_current_emp
        FROM departments
        WHERE name = NEW.department;
        
        -- Check employee count limit
        IF dept_current_emp >= dept_max_emp THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Department has reached maximum employee count';
        END IF;
        
        -- Check budget limit
        IF dept_spending + NEW.salary > dept_budget THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Adding this employee would exceed department budget';
        END IF;
    END$$
    
    CREATE TRIGGER update_department_spending_after_insert
    AFTER INSERT ON employees
    FOR EACH ROW
    BEGIN
        UPDATE departments
        SET current_spending = current_spending + NEW.salary,
            employee_count = employee_count + 1
        WHERE name = NEW.department;
    END$$
    
    CREATE TRIGGER update_department_spending_after_delete
    AFTER DELETE ON employees
    FOR EACH ROW
    BEGIN
        UPDATE departments
        SET current_spending = current_spending - OLD.salary,
            employee_count = employee_count - 1
        WHERE name = OLD.department;
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, budget_triggers_sql, 
                  description="Creating department budget triggers", fetch=False)
    
    # Check initial department status
    df = execute_query(conn, "SELECT * FROM departments",
                      description="Initial department status")
    if df is not None:
        display(df)
    
    # Test adding employees within budget
    test_employees = [
        ('Bob Wilson', 'Sales', 55000.00, '2024-04-05'),
        ('Carol Davis', 'Finance', 80000.00, '2024-05-12')
    ]
    
    for name, dept, salary, hire_date in test_employees:
        insert_sql = f"""
        INSERT INTO employees (name, department, salary, hire_date) 
        VALUES ('{name}', '{dept}', {salary}, '{hire_date}')
        """
        try:
            execute_query(conn, insert_sql, fetch=False)
            print(f"✅ Successfully added {name} to {dept}")
        except Exception as e:
            print(f"❌ Failed to add {name}: {str(e)}")
    
    # Check updated department status
    df = execute_query(conn, "SELECT name, budget, current_spending, employee_count, max_employees FROM departments",
                      description="Updated department status after adding employees")
    if df is not None:
        display(df)
    
    # Test budget limit (should fail)
    print("\n🧪 Testing budget limit (should fail):")
    try:
        execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('Overspending Employee', 'Finance', 200000.00, '2024-06-01')", fetch=False)
        print("❌ Expected this to fail!")
    except Exception as e:
        print(f"✅ Correctly rejected: {str(e)}")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 6: Scheduled Events

Let's create and test scheduled events for automated database maintenance.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Enable event scheduler
    execute_query(conn, "SET GLOBAL event_scheduler = ON", 
                  description="Enabling event scheduler", fetch=False)
    
    # Check event scheduler status
    df = execute_query(conn, "SHOW VARIABLES LIKE 'event_scheduler'",
                      description="Checking event scheduler status")
    if df is not None:
        display(df)
    
    # Create scheduled events
    events_sql = """
    DELIMITER $$
    
    CREATE EVENT daily_audit_cleanup
    ON SCHEDULE EVERY 1 DAY STARTS CURDATE() + INTERVAL 1 DAY
    DO
    BEGIN
        DECLARE deleted_count INT DEFAULT 0;
        
        -- Remove audit logs older than 1 year
        DELETE FROM audit_log
        WHERE operation_time < DATE_SUB(CURDATE(), INTERVAL 1 YEAR);
        
        SET deleted_count = ROW_COUNT();
        
        -- Log the cleanup operation
        INSERT INTO notifications (message, recipient, status)
        VALUES (CONCAT('Daily cleanup completed. Removed ', deleted_count, ' old audit records.'),
                'admin@company.com', 'sent');
    END$$
    
    CREATE EVENT weekly_budget_report
    ON SCHEDULE EVERY 1 WEEK STARTS CURDATE() + INTERVAL 1 DAY
    DO
    BEGIN
        -- Generate budget reports for departments over 90% utilization
        INSERT INTO notifications (message, recipient, status)
        SELECT CONCAT('Budget Alert: ', name, ' department is at ',
                     ROUND((current_spending / budget) * 100, 1), '% capacity'),
               'management@company.com',
               'pending'
        FROM departments
        WHERE (current_spending / budget) > 0.9;
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, events_sql, 
                  description="Creating scheduled events", fetch=False)
    
    # View created events
    df = execute_query(conn, "SHOW EVENTS",
                      description="Viewing created events")
    if df is not None:
        display(df)
    
    # Manually trigger an event for testing (simulate budget alert)
    print("\n🧪 Manually testing budget report event:")
    
    # First, let's create a high-spending department to trigger the alert
    execute_query(conn, "UPDATE departments SET current_spending = budget * 0.95 WHERE name = 'IT'", 
                  description="Setting IT department to 95% budget utilization", fetch=False)
    
    # Now manually execute the event logic
    manual_event_sql = """
    INSERT INTO notifications (message, recipient, status)
    SELECT CONCAT('Budget Alert: ', name, ' department is at ',
                 ROUND((current_spending / budget) * 100, 1), '% capacity'),
           'management@company.com',
           'pending'
    FROM departments
    WHERE (current_spending / budget) > 0.9
    """
    execute_query(conn, manual_event_sql, 
                  description="Manually executing budget alert logic", fetch=False)
    
    # Check notifications
    df = execute_query(conn, "SELECT * FROM notifications WHERE recipient = 'management@company.com' ORDER BY sent_at DESC",
                      description="Checking budget alert notifications")
    if df is not None:
        display(df)
    
    # Test cleanup event manually
    print("\n🧪 Testing cleanup event logic:")
    
    # Add some old audit records for testing
    old_audit_sql = """
    INSERT INTO audit_log (table_name, operation, record_id, new_values, user_name, operation_time) 
    VALUES 
    ('test', 'INSERT', 999, '{"test": "old record"}', 'system', DATE_SUB(CURDATE(), INTERVAL 2 YEAR)),
    ('test', 'INSERT', 998, '{"test": "old record 2"}', 'system', DATE_SUB(CURDATE(), INTERVAL 15 MONTH))
    """
    execute_query(conn, old_audit_sql, 
                  description="Adding old audit records for testing", fetch=False)
    
    # Check old records count
    df = execute_query(conn, "SELECT COUNT(*) as old_records FROM audit_log WHERE operation_time < DATE_SUB(CURDATE(), INTERVAL 1 YEAR)",
                      description="Counting old audit records before cleanup")
    if df is not None:
        old_count = df.iloc[0]['old_records']
        print(f"Old records before cleanup: {old_count}")
    
    # Manually execute cleanup
    execute_query(conn, "DELETE FROM audit_log WHERE operation_time < DATE_SUB(CURDATE(), INTERVAL 1 YEAR)", 
                  description="Executing cleanup logic", fetch=False)
    
    # Check remaining records
    df = execute_query(conn, "SELECT COUNT(*) as remaining_records FROM audit_log",
                      description="Counting remaining audit records after cleanup")
    if df is not None:
        remaining_count = df.iloc[0]['remaining_records']
        print(f"Records after cleanup: {remaining_count}")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 7: Managing Triggers and Events

Let's learn how to view, modify, and remove triggers and events.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # View all triggers
    df = execute_query(conn, "SHOW TRIGGERS",
                      description="Viewing all triggers in the database")
    if df is not None:
        display(df)
    
    # Get detailed trigger information
    df = execute_query(conn, """
    SELECT 
        trigger_name,
        event_manipulation,
        event_object_table,
        action_timing,
        action_order
    FROM information_schema.triggers
    WHERE trigger_schema = DATABASE()
    ORDER BY event_object_table, action_timing, action_order
    """, description="Detailed trigger information from information_schema")
    if df is not None:
        display(df)
    
    # View all events
    df = execute_query(conn, "SHOW EVENTS",
                      description="Viewing all scheduled events")
    if df is not None:
        display(df)
    
    # Get detailed event information
    df = execute_query(conn, """
    SELECT 
        event_name,
        event_definition,
        event_type,
        execute_at,
        interval_value,
        interval_field,
        status,
        created
    FROM information_schema.events
    WHERE event_schema = DATABASE()
    """, description="Detailed event information from information_schema")
    if df is not None:
        display(df)
    
    # Demonstrate trigger execution order
    print("\n🔄 Testing trigger execution order:")
    
    # Create an additional trigger to demonstrate order
    additional_trigger_sql = """
    DELIMITER $$
    
    CREATE TRIGGER additional_salary_trigger
    AFTER UPDATE ON employees
    FOR EACH ROW
    BEGIN
        IF OLD.salary != NEW.salary THEN
            INSERT INTO notifications (message, recipient)
            VALUES (CONCAT('Additional trigger: Salary changed for ', NEW.name),
                    'system@company.com');
        END IF;
    END$$
    
    DELIMITER ;
    """
    execute_query(conn, additional_trigger_sql, 
                  description="Creating additional trigger to demonstrate execution order", fetch=False)
    
    # Test with a salary update
    execute_query(conn, "UPDATE employees SET salary = 78000.00 WHERE name = 'ALICE JOHNSON'", 
                  description="Testing trigger execution with salary update", fetch=False)
    
    # Check notifications to see both triggers fired
    df = execute_query(conn, "SELECT message, recipient FROM notifications ORDER BY sent_at DESC LIMIT 5",
                      description="Checking notifications from multiple triggers")
    if df is not None:
        display(df)
    
    # Demonstrate dropping triggers
    print("\n🗑️ Demonstrating trigger removal:")
    
    execute_query(conn, "DROP TRIGGER IF EXISTS additional_salary_trigger", 
                  description="Dropping the additional trigger", fetch=False)
    
    # Verify trigger was dropped
    df = execute_query(conn, "SHOW TRIGGERS WHERE `table` = 'employees'",
                      description="Verifying trigger was removed")
    if df is not None:
        display(df)
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 8: Advanced Trigger Concepts

Let's explore complex trigger logic and error handling.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    # Create complex business rules trigger
    complex_trigger_sql = """
    DELIMITER $$
    
    CREATE TRIGGER complex_business_rules
    BEFORE INSERT ON employees
    FOR EACH ROW
    BEGIN
        DECLARE dept_name VARCHAR(50);
        DECLARE emp_count INT;
        DECLARE avg_salary DECIMAL(10,2);
        
        -- Get department info
        SELECT name INTO dept_name
        FROM departments
        WHERE name = NEW.department;
        
        -- If department not found, create it automatically
        IF dept_name IS NULL THEN
            BEGIN
                DECLARE EXIT HANDLER FOR SQLEXCEPTION
                BEGIN
                    SIGNAL SQLSTATE '45000'
                    SET MESSAGE_TEXT = 'Could not create department automatically';
                END;
                
                INSERT INTO departments (name, budget, max_employees)
                VALUES (NEW.department, 100000.00, 10);
                
                INSERT INTO notifications (message, recipient)
                VALUES (CONCAT('New department created automatically: ', NEW.department),
                        'admin@company.com');
            END;
        END IF;
        
        -- Business rule: Don't allow more than 3 employees with salary > 100k in same dept
        SELECT COUNT(*), AVG(salary)
        INTO emp_count, avg_salary
        FROM employees
        WHERE department = NEW.department AND salary > 100000;
        
        IF emp_count >= 3 THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = 'Department already has 3 high-salary employees';
        END IF;
        
        -- Business rule: New employee salary shouldn't be more than 150% of dept average
        IF NEW.salary > avg_salary * 1.5 AND avg_salary > 0 THEN
            INSERT INTO notifications (message, recipient)
            VALUES (CONCAT('High salary alert: ', NEW.name, ' salary ($', NEW.salary,
                          ') is 150% above department average ($', FORMAT(avg_salary, 2), ')'),
                    'ceo@company.com');
        END IF;
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, complex_trigger_sql, 
                  description="Creating complex business rules trigger", fetch=False)
    
    # Test automatic department creation
    print("\n🧪 Testing automatic department creation:")
    
    try:
        execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('New Dept Employee', 'Research', 65000.00, '2024-07-01')", 
                      description="Testing automatic department creation", fetch=False)
        print("✅ Department created automatically")
    except Exception as e:
        print(f"❌ Failed: {str(e)}")
    
    # Check if department was created
    df = execute_query(conn, "SELECT * FROM departments WHERE name = 'Research'",
                      description="Checking if Research department was created")
    if df is not None:
        display(df)
    
    # Check notifications
    df = execute_query(conn, "SELECT * FROM notifications WHERE recipient = 'admin@company.com' ORDER BY sent_at DESC",
                      description="Checking admin notifications")
    if df is not None:
        display(df)
    
    # Test high salary alert
    print("\n🧪 Testing high salary alert:")
    
    # First add some employees to establish an average
    execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('Avg Employee 1', 'Research', 50000.00, '2024-07-02')", fetch=False)
    execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('Avg Employee 2', 'Research', 55000.00, '2024-07-03')", fetch=False)
    
    # Now add a high-salary employee (should trigger alert)
    execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('High Salary Employee', 'Research', 150000.00, '2024-07-04')", 
                  description="Testing high salary alert trigger", fetch=False)
    
    # Check CEO notifications
    df = execute_query(conn, "SELECT * FROM notifications WHERE recipient = 'ceo@company.com' ORDER BY sent_at DESC",
                      description="Checking CEO notifications for high salary alert")
    if df is not None:
        display(df)
    
    # Test business rule violation (should fail)
    print("\n🧪 Testing business rule violation (should fail):")
    
    # Add more high-salary employees to reach the limit
    execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('High Salary 2', 'Research', 120000.00, '2024-07-05')", fetch=False)
    execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('High Salary 3', 'Research', 130000.00, '2024-07-06')", fetch=False)
    
    # This should fail (4th high-salary employee)
    try:
        execute_query(conn, "INSERT INTO employees (name, department, salary, hire_date) VALUES ('High Salary 4', 'Research', 140000.00, '2024-07-07')", fetch=False)
        print("❌ Expected this to fail!")
    except Exception as e:
        print(f"✅ Correctly rejected: {str(e)}")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Exercise 9: Best Practices and Troubleshooting

Let's explore best practices and common troubleshooting techniques.

In [ ]:
# Connect to database
conn = create_connection()

if conn:
    print("📋 Exercise 9: Best Practices and Troubleshooting")
    print("\n✅ Best Practices Demonstrated:")
    print("1. Keep triggers simple and focused")
    print("2. Use appropriate trigger types (BEFORE vs AFTER)")
    print("3. Handle errors gracefully with SIGNAL")
    print("4. Document trigger purposes")
    print("5. Test thoroughly before deployment")
    
    # Demonstrate trigger performance monitoring
    print("\n📊 Performance Monitoring:")
    
    # Check trigger count per table
    df = execute_query(conn, """
    SELECT event_object_table, COUNT(*) as trigger_count
    FROM information_schema.triggers
    WHERE trigger_schema = DATABASE()
    GROUP BY event_object_table
    """, description="Checking trigger count per table")
    if df is not None:
        display(df)
    
    # Show current database size and activity
    df = execute_query(conn, "SHOW TABLE STATUS LIKE 'employees'",
                      description="Checking employees table status")
    if df is not None:
        display(df[['Name', 'Rows', 'Avg_row_length', 'Data_length']])
    
    # Demonstrate error handling in triggers
    print("\n🚨 Error Handling Demonstration:")
    
    # Create a trigger with comprehensive error handling
    error_handling_trigger_sql = """
    DELIMITER $$
    
    CREATE TRIGGER robust_error_handling
    BEFORE UPDATE ON employees
    FOR EACH ROW
    BEGIN
        DECLARE error_message TEXT DEFAULT '';
        
        -- Validate input data
        IF NEW.salary IS NULL OR NEW.salary <= 0 THEN
            SET error_message = CONCAT(error_message, 'Invalid salary. ');
        END IF;
        
        IF NEW.name IS NULL OR LENGTH(TRIM(NEW.name)) = 0 THEN
            SET error_message = CONCAT(error_message, 'Name cannot be empty. ');
        END IF;
        
        IF NEW.hire_date > CURDATE() THEN
            SET error_message = CONCAT(error_message, 'Hire date cannot be in future. ');
        END IF;
        
        -- If any errors, signal them
        IF LENGTH(error_message) > 0 THEN
            SIGNAL SQLSTATE '45000'
            SET MESSAGE_TEXT = error_message;
        END IF;
        
        -- Log the update attempt
        INSERT INTO audit_log (table_name, operation, record_id, 
                              old_values, new_values, user_name)
        VALUES ('employees', 'UPDATE_ATTEMPT', NEW.id,
                JSON_OBJECT('name', OLD.name, 'salary', OLD.salary),
                JSON_OBJECT('name', NEW.name, 'salary', NEW.salary),
                USER());
    END$$
    
    DELIMITER ;
    """
    
    execute_query(conn, error_handling_trigger_sql, 
                  description="Creating robust error handling trigger", fetch=False)
    
    # Test error handling
    error_tests = [
        ("Valid update", "UPDATE employees SET salary = 85000.00 WHERE name = 'JOHN DOE'", True),
        ("Invalid salary", "UPDATE employees SET salary = -1000 WHERE name = 'JOHN DOE'", False),
        ("Empty name", "UPDATE employees SET name = '' WHERE name = 'JOHN DOE'", False)
    ]
    
    for test_name, test_sql, should_succeed in error_tests:
        print(f"\n🧪 Testing: {test_name}")
        try:
            execute_query(conn, test_sql, fetch=False)
            if should_succeed:
                print("✅ Succeeded as expected")
            else:
                print("❌ Expected this to fail!")
        except Exception as e:
            if not should_succeed:
                print(f"✅ Correctly rejected: {str(e)}")
            else:
                print(f"❌ Unexpected failure: {str(e)}")
    
    # Check audit log for update attempts
    df = execute_query(conn, "SELECT * FROM audit_log WHERE operation = 'UPDATE_ATTEMPT' ORDER BY operation_time DESC",
                      description="Checking audit log for update attempts")
    if df is not None:
        display(df)
    
    # Cleanup demonstration
    print("\n🧹 Cleanup: Removing test triggers")
    
    cleanup_triggers = [
        "DROP TRIGGER IF EXISTS robust_error_handling",
        "DROP EVENT IF EXISTS daily_audit_cleanup",
        "DROP EVENT IF EXISTS weekly_budget_report"
    ]
    
    for cleanup_sql in cleanup_triggers:
        execute_query(conn, cleanup_sql, fetch=False)
    
    print("✅ Cleanup completed")
    
    conn.close()
else:
    print("❌ Cannot proceed without database connection")

## Summary

In this lab, you learned:

1. **Trigger Fundamentals**: BEFORE/AFTER triggers, INSERT/UPDATE/DELETE events
2. **Audit Triggers**: Automatic logging of all database changes
3. **Validation Triggers**: Data validation and business rule enforcement
4. **Business Logic**: Complex triggers with conditional logic and error handling
5. **Scheduled Events**: Automated database maintenance and reporting
6. **Management**: Viewing, modifying, and removing triggers and events
7. **Best Practices**: Performance considerations and troubleshooting

### Key Takeaways:
- Triggers automate database operations and maintain data integrity
- BEFORE triggers validate and modify data before changes
- AFTER triggers respond to successful operations
- Events schedule recurring database tasks
- Proper error handling prevents data corruption
- Monitor trigger performance and avoid complex logic

### Next Steps:
1. **Lab 14**: Performance Tuning and Optimization
2. **Lab 15**: Database Security and User Management
3. **Lab 16**: Backup, Recovery, and High Availability

### Advanced Topics to Explore:
- **INSTEAD OF triggers** (for views in other databases)
- **Compound triggers** (Oracle-specific)
- **Cross-database triggers**
- **Event chaining and dependencies**
- **Trigger performance optimization**

Remember: Triggers are powerful but can impact performance. Use them judiciously and always test thoroughly in development environments!